In [3]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset
from tensorflow.keras.datasets import imdb
from tensorflow.keras.preprocessing import sequence
import numpy as np

In [5]:
# Check if CUDA is available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [6]:
device

device(type='cuda')

In [8]:
# Set random seed for reproducibility
torch.manual_seed(42)

# Load the IMDB dataset (same as in TensorFlow implementation)
max_features = 10000  # vocabulary size
(X_train, y_train), (X_test, y_test) = imdb.load_data(num_words=max_features)

In [9]:
# Pad sequences to a max length
max_len = 500
X_train = sequence.pad_sequences(X_train, maxlen=max_len)
X_test = sequence.pad_sequences(X_test, maxlen=max_len)

In [10]:
# Convert numpy arrays to PyTorch tensors and move to device (GPU/CPU)
X_train_tensor = torch.tensor(X_train, dtype=torch.long).to(device)
X_test_tensor = torch.tensor(X_test, dtype=torch.long).to(device)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32).to(device)
y_test_tensor = torch.tensor(y_test, dtype=torch.float32).to(device)

In [11]:
# Prepare DataLoader for training and testing datasets
train_data = TensorDataset(X_train_tensor, y_train_tensor)
test_data = TensorDataset(X_test_tensor, y_test_tensor)

train_loader = DataLoader(train_data, batch_size=32, shuffle=True)
test_loader = DataLoader(test_data, batch_size=32, shuffle=False)

In [12]:
# Define the RNN Model using PyTorch
class SimpleRNN(nn.Module):
    def __init__(self, input_size, embedding_dim, hidden_size, output_size):
        super(SimpleRNN, self).__init__()
        self.embedding = nn.Embedding(input_size, embedding_dim)
        self.rnn = nn.RNN(embedding_dim, hidden_size, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = self.embedding(x)
        out, _ = self.rnn(x)
        out = out[:, -1, :]  # Get the output of the last time step
        out = self.fc(out)
        out = self.sigmoid(out)
        return out

In [13]:
# Initialize the model, loss, and optimizer
model = SimpleRNN(max_features, 128, 128, 1).to(device)  # Move model to device
criterion = nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [26]:
# Training the model
epochs = 10
for epoch in range(epochs):
    model.train()
    running_loss = 0.0
    correct_predictions = 0
    total_predictions = 0

    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)  # Move data to device
        optimizer.zero_grad()
        outputs = model(inputs).squeeze(1)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

        # Calculate accuracy
        predicted = outputs > 0.5
        correct_predictions += (predicted == labels).sum().item()
        total_predictions += labels.size(0)

    train_loss = running_loss / len(train_loader)
    train_accuracy = correct_predictions / total_predictions

    # Validation
    model.eval()
    val_loss = 0.0
    correct_predictions = 0
    total_predictions = 0

    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.to(device)  # Move data to device
            outputs = model(inputs).squeeze(1)
            loss = criterion(outputs, labels)
            val_loss += loss.item()

            # Calculate accuracy
            predicted = outputs > 0.5
            correct_predictions += (predicted == labels).sum().item()
            total_predictions += labels.size(0)

    val_loss = val_loss / len(test_loader)
    val_accuracy = correct_predictions / total_predictions

    print(f"Epoch {epoch+1}/{epochs}")
    print(f"Train Loss: {train_loss:.4f}, Train Accuracy: {train_accuracy:.4f}")
    print(f"Validation Loss: {val_loss:.4f}, Validation Accuracy: {val_accuracy:.4f}")

Epoch 1/10
Train Loss: 0.4959, Train Accuracy: 0.7413
Validation Loss: 0.7085, Validation Accuracy: 0.6246
Epoch 2/10
Train Loss: 0.5015, Train Accuracy: 0.7334
Validation Loss: 0.6928, Validation Accuracy: 0.6153
Epoch 3/10
Train Loss: 0.5077, Train Accuracy: 0.7323
Validation Loss: 0.7149, Validation Accuracy: 0.6272
Epoch 4/10
Train Loss: 0.4908, Train Accuracy: 0.7439
Validation Loss: 0.7071, Validation Accuracy: 0.6347
Epoch 5/10
Train Loss: 0.4837, Train Accuracy: 0.7495
Validation Loss: 0.7005, Validation Accuracy: 0.6378
Epoch 6/10
Train Loss: 0.4805, Train Accuracy: 0.7510
Validation Loss: 0.7000, Validation Accuracy: 0.6326
Epoch 7/10
Train Loss: 0.4873, Train Accuracy: 0.7458
Validation Loss: 0.7007, Validation Accuracy: 0.6315
Epoch 8/10
Train Loss: 0.4786, Train Accuracy: 0.7513
Validation Loss: 0.7007, Validation Accuracy: 0.6316
Epoch 9/10
Train Loss: 0.4791, Train Accuracy: 0.7531
Validation Loss: 0.7051, Validation Accuracy: 0.6334
Epoch 10/10
Train Loss: 0.4789, Train

In [16]:
# Save the model
torch.save(model.state_dict(), 'imdb_simple_rnn.pth')

In [17]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [18]:
# Sample input text (a review)
input_text = "This movie was amazing! The acting was superb."

In [19]:
# Tokenize the input text (using the same tokenizer settings as during training)
tokenizer = Tokenizer(num_words=max_features)
tokenizer.fit_on_texts([input_text])  # Fit the tokenizer on the input text

In [20]:
# Convert the input text to a sequence of integers
input_sequence = tokenizer.texts_to_sequences([input_text])

In [21]:
# Pad the sequence to the same length as the training data
input_padded = pad_sequences(input_sequence, maxlen=max_len)

In [22]:
# Convert to a PyTorch tensor
input_tensor = torch.tensor(input_padded, dtype=torch.long)

In [23]:
# Ensure that the model is in evaluation mode
model.eval()

SimpleRNN(
  (embedding): Embedding(10000, 128)
  (rnn): RNN(128, 128, batch_first=True)
  (fc): Linear(in_features=128, out_features=1, bias=True)
  (sigmoid): Sigmoid()
)

In [24]:
# Make the prediction (move input tensor to GPU if available)
with torch.no_grad():  # No need for gradient calculation during inference
    if torch.cuda.is_available():
        input_tensor = input_tensor.cuda()
        model = model.cuda()

    # Get the model's prediction
    output = model(input_tensor)

    # Apply the sigmoid function to the output (since we used BCELoss)
    prediction = output.squeeze().item()

    # Print the predicted sentiment
    if prediction > 0.5:
        print("Prediction: Positive")
    else:
        print("Prediction: Negative")

Prediction: Negative


What ??